In [1]:
import joblib
import os
import numpy as np
import matplotlib.pyplot as plt
import quaternion as q

/home/obin/.local/lib/python3.5/site-packages/quaternion/calculus.py:475: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from scipy, which means that derivatives
and integrals will use less accurate finite-differencing
techniques.  You may want to install scipy.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)


In [2]:
def slice_data_by_index(data, s_idx, e_idx):
    nu_data = dict()
    for key in data.keys():
        nu_data[key] = data[key][s_idx:e_idx+1]
    return nu_data

def arr_quat_to_rots(quats):
    ret = q.as_rotation_vector(q.as_quat_array(quats))
    ret = ret.reshape(-1,3)
    ret = ret[:,1]-3.14
    ret[np.where(ret > 3.14)] -= 6.28
    ret[np.where(ret < -3.14)] += 6.28
    return ret

quaternion(3.26794896538138e-07, 0, 0, 0.999999999999947)

In [9]:
demo_data['rotation']
print()

[array([0.79935586, 0.        , 0.60085785, 0.        ]),
 array([0.89154977, 0.        , 0.45292279, 0.        ]),
 array([0.95665431, 0.        , 0.29122585, 0.        ]),
 array([0.99269146, 0.        , 0.12068019, 0.        ]),
 array([0.99269146, 0.        , 0.12068019, 0.        ]),
 array([0.99269146, 0.        , 0.12068019, 0.        ]),
 array([0.99269146, 0.        , 0.12068019, 0.        ]),
 array([0.99269146, 0.        , 0.12068019, 0.        ]),
 array([0.99269146, 0.        , 0.12068019, 0.        ]),
 array([0.99269146, 0.        , 0.12068019, 0.        ]),
 array([0.99269146, 0.        , 0.12068019, 0.        ]),
 array([ 0.99856615,  0.        , -0.05353228,  0.        ]),
 array([ 0.99856615,  0.        , -0.05353228,  0.        ]),
 array([ 0.99856615,  0.        , -0.05353228,  0.        ]),
 array([ 0.99856615,  0.        , -0.05353228,  0.        ]),
 array([ 0.99856615,  0.        , -0.05353228,  0.        ]),
 array([ 0.99856615,  0.        , -0.05353228,  0.  

In [18]:
demo

'../habitat-api/demo_data/s8pcmisQ38h/DEMON/s8pcmisQ38h_158._env0dat.gz'

In [15]:
DATA_PATH = '../habitat-api/demo_data'

SAVE_PATH = 'preprocessed_habitat_data2/'
if not os.path.exists(SAVE_PATH): os.mkdir(SAVE_PATH)
    
if not os.path.exists(SAVE_PATH+'DEMON/') : os.mkdir(SAVE_PATH+'DEMON/')
if not os.path.exists(SAVE_PATH+'NOISE/') : os.mkdir(SAVE_PATH+'NOISE/')
space_list = os.listdir(DATA_PATH)
#space_list.remove('saved_envs')

In [5]:
space_list.index('XcA2TqTSSAj')

ValueError: 'XcA2TqTSSAj' is not in list

In [17]:
demo_length = 30
noise_lookup_bw = 10
noise_data_num = 0
for space in space_list: 
    try:
        demonstrations = [os.path.join(DATA_PATH,space,'DEMON',x) for x in os.listdir(os.path.join(DATA_PATH,space,'DEMON'))]
        noised_demon = [os.path.join(DATA_PATH,space,'NOISE',x) for x in os.listdir(os.path.join(DATA_PATH,space,'NOISE'))]
    except:
        continue
    print('='*10 + space + '='*10)
    for demo_id, demo in enumerate(demonstrations) :
        episode_name = demo[demo.find('DEMON')+6:]
        demo_data = joblib.load(demo)
        noise_datas = []
        noise_demo = demo.replace('DEMON','NOISE')
        for i in range(3):
            idx = demo.find('dat.gz')
            noise_demo = noise_demo[:idx-1] + '{}dat.gz'.format(i)
            try: 
                noise_datas.append(joblib.load(noise_demo))
            except : 
                print('load {} failed'.format(noise_demo))
                continue

        slicable_num = int(len(demo_data['position'])/demo_length)
        saved_noised = 0
        fail_case = {'end_pos_far': 0 , 'start_pos_far': 0, 'long': 0 }
        if len(demo_data['position']) < demo_length : continue
        for i in range(slicable_num):
            start_pos = demo_data['position'][demo_length*i]
            start_quat = demo_data['rotation'][demo_length*i]
            end_pos = demo_data['position'][demo_length*(i+1)-1]
            end_quat = demo_data['rotation'][demo_length*(i+1)-1]

            demo_file_name = os.path.join(SAVE_PATH,'DEMON',episode_name).replace('._env0dat.gz','_num{}_{}.dat.gz'.format(i,0))
            joblib.dump(slice_data_by_index(demo_data, demo_length*i, demo_length*(i+1)-1),demo_file_name)
            print(demo_file_name)
            for noise_id, noise_data in enumerate(noise_datas):
                s_norms = np.linalg.norm(noise_data['position'] - start_pos,axis=1)
                min_dist_start_idx = np.argmin(s_norms) if i > 0 else 0
                
                norms = np.linalg.norm(noise_data['position'] - end_pos,axis=1)
                min_dist_idx = np.argmin(norms)
                
                rots = arr_quat_to_rots(noise_data['rotation'])
                start_rots = arr_quat_to_rots(start_quat)
                end_rots = arr_quat_to_rots(end_quat)
                
                start_rot_norms = rots - start_rots
                end_rot_norms = rots - end_rots
                start_rot_norms[np.where(start_rot_norms > 3.14)] -= 6.28
                start_rot_norms[np.where(start_rot_norms < -3.14)] += 6.28
                end_rot_norms[np.where(end_rot_norms > 3.14)] -= 6.28
                end_rot_norms[np.where(end_rot_norms < -3.14)] += 6.28
                
                if (min_dist_idx - demo_length*i) > 50 : 
                    fail_case['long'] += 1
                    continue
                if norms[min_dist_idx] < 0.4 and abs(end_rot_norms[min_dist_idx]) < 0.35 : 
                    if i > 0 and abs(start_rot_norms[min_dist_start_idx]) > 0.35 : 
                        fail_case['start_pos_far'] += 1
                        continue
                    noise_data_num += 1
                    saved_noised += 1
                    noise_ep_name = episode_name.replace('._env0dat.gz', '_num{}_{}.dat.gz'.format(i,noise_id))
                    joblib.dump(slice_data_by_index(noise_data,min_dist_start_idx,min_dist_idx), os.path.join(SAVE_PATH,'NOISE', noise_ep_name))
                    print(os.path.join(SAVE_PATH,'NOISE', noise_ep_name))
                    #print('you should check',norms[min_dist_idx],end_rot_norms[min_dist_idx])
                    #view_img = np.zeros([256, 256*4, 3],dtype=np.uint8)
                    #view_img[:,:256,:] = demo_data['rgb'][demo_length*i]
                    #view_img[:,256:256*2,:] = noise_data['rgb'][min_dist_start_idx]
                    #view_img[:,256*2:256*3,:] = demo_data['rgb'][demo_length*(i+1)-1]
                    #view_img[:, 256*3:,:] = noise_data['rgb'][min_dist_idx]
                    #plt.imshow(view_img)
                    #plt.show()
                    
                else:
                    fail_case['end_pos_far'] += 1
                    #print("CHECK")
                    #view_img = np.zeros([256, 256*4, 3],dtype=np.uint8)
                    #view_img[:,:256,:] = demo_data['rgb'][0]
                    #view_img[:,256:256*2,:] = noise_data['rgb'][0]
                    #view_img[:,256*2:256*3,:] = demo_data['rgb'][demo_length*(i+1)-1]
                    #view_img[:, 256*3:,:] = noise_data['rgb'][min_dist_idx]
                    #plt.imshow(view_img)
                    #plt.show()
                    pass

        print('[%03d/%03d] ep process done len %d - %d/%d noised data saved'%(
            demo_id, len(demonstrations), len(demo_data['position']), saved_noised, slicable_num * len(noise_datas)))
        #print('\t too long {}, end_pos_far {}, start_pos_far {}'.format(fail_case['long'],fail_case['end_pos_far'],fail_case['start_pos_far']))

==========Pm6F8kyY3z2==========
load ../habitat-api/demo_data/Pm6F8kyY3z2/NOISE/Pm6F8kyY3z2_274._env0dat.gz failed
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_274_num0_0.dat.gz
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_274_num1_0.dat.gz
[000/287] ep process done len 80 - 0/4 noised data saved
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_131_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_131_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_131_num0_1.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_131_num0_2.dat.gz
[001/287] ep process done len 35 - 3/3 noised data saved
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_257_num0_0.dat.gz
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_257_num1_0.dat.gz
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_257_num2_0.dat.gz
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_257_num3_0.dat.gz
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_257_num4_0.dat.gz
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_257_num5_0.dat.gz
preprocessed_ha

load ../habitat-api/demo_data/Pm6F8kyY3z2/NOISE/Pm6F8kyY3z2_173._env1dat.gz failed
load ../habitat-api/demo_data/Pm6F8kyY3z2/NOISE/Pm6F8kyY3z2_000._env2dat.gz failed
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_000_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_000_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_000_num0_1.dat.gz
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_000_num1_0.dat.gz
[042/287] ep process done len 60 - 2/4 noised data saved
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_123_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_123_num0_1.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_123_num0_2.dat.gz
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_123_num1_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_123_num1_1.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_123_num1_2.dat.gz
[044/287] ep process done len 66 - 4/6 noised data saved
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_150_num0_0.dat.gz
preprocessed_habitat_data2/

preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_268_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_268_num0_1.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_268_num0_2.dat.gz
[078/287] ep process done len 34 - 2/3 noised data saved
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_136_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_136_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_136_num0_1.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_136_num0_2.dat.gz
[079/287] ep process done len 31 - 3/3 noised data saved
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_270_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_270_num0_1.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_270_num0_2.dat.gz
[081/287] ep process done len 37 - 2/3 noised data saved
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_256_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_256_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_256_num0_1.dat.gz
preprocess

preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_119_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_119_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_119_num0_1.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_119_num0_2.dat.gz
[121/287] ep process done len 49 - 3/3 noised data saved
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_020_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_020_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_020_num0_1.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_020_num0_2.dat.gz
[122/287] ep process done len 57 - 3/3 noised data saved
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_142_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_142_num0_2.dat.gz
[123/287] ep process done len 53 - 1/3 noised data saved
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_253_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_253_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_253_num0_1.dat.gz
preprocess

preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_227_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_227_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_227_num0_1.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_227_num0_2.dat.gz
[162/287] ep process done len 53 - 3/3 noised data saved
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_223_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_223_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_223_num0_1.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_223_num0_2.dat.gz
[163/287] ep process done len 46 - 3/3 noised data saved
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_190_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_190_num0_1.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_190_num0_2.dat.gz
[165/287] ep process done len 57 - 2/3 noised data saved
preprocessed_habitat_data2/DEMON/Pm6F8kyY3z2_052_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/Pm6F8kyY3z2_052_num0_1.dat.gz
[167/287] 

...

preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_048_num0_2.dat.gz
[234/291] ep process done len 50 - 2/3 noised data saved
preprocessed_habitat_data2/DEMON/ur6pFq6Qu1A_013_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_013_num0_1.dat.gz
preprocessed_habitat_data2/DEMON/ur6pFq6Qu1A_013_num1_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_013_num1_1.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_013_num1_2.dat.gz
preprocessed_habitat_data2/DEMON/ur6pFq6Qu1A_013_num2_0.dat.gz
[235/291] ep process done len 107 - 3/9 noised data saved
load ../habitat-api/demo_data/ur6pFq6Qu1A/NOISE/ur6pFq6Qu1A_212._env2dat.gz failed
preprocessed_habitat_data2/DEMON/ur6pFq6Qu1A_212_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_212_num0_1.dat.gz
preprocessed_habitat_data2/DEMON/ur6pFq6Qu1A_212_num1_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_212_num1_0.dat.gz
preprocessed_habitat_data2/DEMON/ur6pFq6Qu1A_212_num2_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_2

preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_073_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_073_num0_1.dat.gz
preprocessed_habitat_data2/DEMON/ur6pFq6Qu1A_073_num1_0.dat.gz
preprocessed_habitat_data2/DEMON/ur6pFq6Qu1A_073_num2_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_073_num2_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_073_num2_1.dat.gz
[255/291] ep process done len 90 - 4/6 noised data saved
preprocessed_habitat_data2/DEMON/ur6pFq6Qu1A_064_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_064_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_064_num0_1.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_064_num0_2.dat.gz
[256/291] ep process done len 51 - 3/3 noised data saved
preprocessed_habitat_data2/DEMON/ur6pFq6Qu1A_284_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_284_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_284_num0_1.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_284_num0_2.dat.gz
prep

preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_005_num2_1.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_005_num2_2.dat.gz
[271/291] ep process done len 111 - 8/9 noised data saved
preprocessed_habitat_data2/DEMON/ur6pFq6Qu1A_232_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_232_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_232_num0_2.dat.gz
preprocessed_habitat_data2/DEMON/ur6pFq6Qu1A_232_num1_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_232_num1_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_232_num1_2.dat.gz
preprocessed_habitat_data2/DEMON/ur6pFq6Qu1A_232_num2_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_232_num2_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_232_num2_1.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_232_num2_2.dat.gz
[272/291] ep process done len 107 - 7/9 noised data saved
preprocessed_habitat_data2/DEMON/ur6pFq6Qu1A_038_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_038_num0_0.dat.gz
pr

preprocessed_habitat_data2/DEMON/ur6pFq6Qu1A_124_num1_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_124_num1_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_124_num1_1.dat.gz
[289/291] ep process done len 82 - 4/4 noised data saved
preprocessed_habitat_data2/DEMON/ur6pFq6Qu1A_150_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_150_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/ur6pFq6Qu1A_150_num0_1.dat.gz
[290/291] ep process done len 40 - 2/3 noised data saved
==========V2XKFyX4ASd==========
load ../habitat-api/demo_data/V2XKFyX4ASd/NOISE/V2XKFyX4ASd_290._env2dat.gz failed
preprocessed_habitat_data2/DEMON/V2XKFyX4ASd_290_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/V2XKFyX4ASd_290_num0_0.dat.gz
preprocessed_habitat_data2/NOISE/V2XKFyX4ASd_290_num0_1.dat.gz
[001/284] ep process done len 47 - 2/2 noised data saved
preprocessed_habitat_data2/DEMON/V2XKFyX4ASd_051_num0_0.dat.gz
preprocessed_habitat_data2/DEMON/V2XKFyX4ASd_051_num1_0.dat.gz
preprocessed_habitat_

KeyboardInterrupt: 

In [8]:
demo_data['position']

[array([ 6.42250109,  3.35230613, -2.07628202]),
 array([ 6.42250109,  3.35230613, -2.07628202]),
 array([ 6.42250109,  3.35230613, -2.07628202]),
 array([ 6.42250109,  3.35230613, -2.07628202]),
 array([ 6.42250109,  3.35230613, -2.07628202]),
 array([ 6.30960512,  3.35230613, -1.79833508]),
 array([ 6.19670916,  3.35230613, -1.52038813]),
 array([ 6.08381319,  3.35230613, -1.24244118]),
 array([ 5.97091722,  3.35230613, -0.96449423]),
 array([ 5.85802126,  3.35230613, -0.68654728]),
 array([ 5.85802126,  3.35230613, -0.68654728])]

In [20]:
valid_space_list = []
valid_space_num = 0
for space in space_list : 
    try:
        demonstrations = [os.path.join(DATA_PATH,space,'DEMON',x) for x in os.listdir(os.path.join(DATA_PATH,space,'DEMON'))]
        noised_demon = [os.path.join(DATA_PATH,space,'NOISE',x) for x in os.listdir(os.path.join(DATA_PATH,space,'NOISE'))]
        valid_space_list.append(space)
        print(space)
        valid_space_num += 1
    except:
        continue
        
valid_space_num

Pm6F8kyY3z2
HxpKQynjfin
1LXtFkjw3qL
8WUmhLawc2A
YmJkqBEsHnH
e9zR4mvMWw7
i5noydFURQK
29hnd4uzFmX
XcA2TqTSSAj
Vvot9Ly1tCj
cV4RVeZvu5T
82sE5b5pLXE
5q7pvUzZiYa
JmbYfDe2QKZ
kEZ7cmS4wCh
Uxmj2M2itWa
jh4fc5c5qoQ
qoiz87JEwZ2
ac26ZMwG7aT
ur6pFq6Qu1A
V2XKFyX4ASd
1pXnuDYAj8r


22

In [22]:
train_set_space = valid_space_list[:20]
valid_set_space = valid_space_list[20:]

# Split Train and Valid

In [18]:
data_dir_list = [os.path.join(SAVE_PATH,'NOISE',x) for x in os.listdir(os.path.join(SAVE_PATH,'NOISE'))]

np.random.shuffle(data_dir_list)
total_num = len(data_dir_list)
train_num = int(total_num * 0.9)
valid_num = total_num - train_num
print(total_num, train_num, valid_num)

18517 16665 1852


In [24]:
data_dir_list[0].split('/')[2].split('_')[0]

'aayBHfsNo7d'

In [23]:
datum

NameError: name 'datum' is not defined

In [25]:
for datum in data_dir_list:
    space = datum.split('/')[2].split('_')[0]
    if space in train_set_space : 
        new_file_name = datum.replace('preprocessed_habitat_data2/NOISE/','preprocessed_habitat_data2/train/')
        os.rename(datum,new_file_name)
    elif space in valid_set_space:
        new_file_name = datum.replace('preprocessed_habitat_data2/NOISE/','preprocessed_habitat_data2/valid/')
        os.rename(datum,new_file_name)
    else : 
        print('error!')

error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!

error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!

error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!

error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!

error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!
error!

In [9]:
for datum in data_dir_list[:train_num]:
    new_file_name = datum.replace('preprocessed_habitat_data/NOISE/','preprocessed_habitat_data/train/')
    os.rename(datum,new_file_name)
for datum in data_dir_list[train_num:]:
    new_file_name = datum.replace('preprocessed_habitat_data/NOISE/','preprocessed_habitat_data/valid/')
    os.rename(datum,new_file_name)

In [162]:
end_rot_norms

array([0.        , 0.41887902, 0.6981317 , 1.11701071, 1.3962634 ,
       1.81514241, 2.23402144, 2.51553778, 2.9318629 , 3.21111556,
       3.14130239, 3.07148923, 3.14130239, 3.07148923, 3.00167606,
       3.28092885, 3.32021909, 3.25040595, 2.831527  , 3.22904662,
       2.81016759, 3.08942023, 3.01960702, 3.29885977, 3.69250928,
       3.62269606, 3.96206544, 3.8922523 , 3.96206544, 4.03187863,
       4.1016918 , 4.17150496, 4.24131813, 3.96206542, 3.89225227,
       3.82243911, 4.24131813, 3.82243911, 4.10169182, 4.03187864,
       3.61299957, 3.19412052, 2.77524154])

In [97]:
nu_data = slice_data_by_index(demo_data,28)

# Data Check

In [1]:
import joblib
import os
import numpy as np
import matplotlib.pyplot as plt
import quaternion as q
import IPython
from IPython.display import display, clear_output
import matplotlib.animation as animation
import cv2

/home/obin/.local/lib/python3.5/site-packages/quaternion/calculus.py:475: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from scipy, which means that derivatives
and integrals will use less accurate finite-differencing
techniques.  You may want to install scipy.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)


In [2]:
DATA_DIR = 'preprocessed_habitat_data'

train_list = [os.path.join(DATA_DIR,'train',x) for x in os.listdir(os.path.join(DATA_DIR,'train'))]
valid_list = [os.path.join(DATA_DIR,'valid',x) for x in os.listdir(os.path.join(DATA_DIR,'valid'))]

train_num = len(train_list)
valid_num = len(valid_list)
print('train_num {}, valid_num {}'.format(train_num, valid_num))
batch_size = 1
action_dim = 4
num_batches = int(train_num/batch_size)
print('num_batches {}'.format(num_batches))


def load_data(data_path):
    noise_data = joblib.load(data_path)
    mode = 'train' if 'train' in str(data_path) else 'valid'
    ref_data_name = str(data_path).replace(mode, 'DEMON')
    ref_data_name = ref_data_name[:ref_data_name.find('.dat.gz')-1] + '0.dat.gz'
    ref_data = joblib.load(str(ref_data_name))
    print(data_path)
    print(ref_data_name)
    noise_rgb  = np.array(noise_data['rgb'],dtype=np.float32)/255.0 * 2 - 1
    ref_rgb = np.array(ref_data['rgb'],dtype=np.float32)/255.0 * 2 - 1
    noise_action = np.eye(action_dim)[noise_data['action']]
    ref_action = np.eye(action_dim)[ref_data['action']]
    
    result = [ref_rgb, ref_action, noise_rgb, noise_action]
    return [np.array(x, dtype=np.float32) for x in result ]


train_num 12915, valid_num 1436
num_batches 12915


In [11]:
error_list = []

In [14]:
#%matplotlib inline

ims = []
fig = plt.figure()
for i,t in enumerate(train_list):
    [ref_rgb, ref_action, noise_rgb, noise_action] = load_data(t)
    print(i)
    for i in range(max(len(ref_rgb), len(noise_rgb))):
        view_image = np.zeros([256, 256*2, 3])
        view_image[:,:256,:] = (ref_rgb[i]+1)/2. if i < len(ref_rgb) else (ref_rgb[len(ref_rgb)-1]+1)/2.
        view_image[:,256:,:] = (noise_rgb[i]+1)/2. if i < len(noise_rgb) else (noise_rgb[len(noise_rgb)-1]+1)/2.
        cv2.imshow('v',view_image)
        cv2.waitKey(20)
        #im = plt.imshow(view_image,animated=True)
        #ims.append([im])
    #ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,repeat_delay=1000)
    #plt.pause(5)
    #plt.show()
    key = input('okay?')
    if 'key' == 'y' : continue
    else : error_list.append(t)
    #break




preprocessed_habitat_data/train/i5noydFURQK_060_2.dat.gz
preprocessed_habitat_data/DEMON/i5noydFURQK_060_0.dat.gz
0
okay?n
preprocessed_habitat_data/train/s8pcmisQ38h_015_3.dat.gz
preprocessed_habitat_data/DEMON/s8pcmisQ38h_015_0.dat.gz
1
okay?y
preprocessed_habitat_data/train/SN83YJsR3w2_143_0.dat.gz
preprocessed_habitat_data/DEMON/SN83YJsR3w2_143_0.dat.gz
2
okay?y
preprocessed_habitat_data/train/VVfe2KiqLaN_024_2.dat.gz
preprocessed_habitat_data/DEMON/VVfe2KiqLaN_024_0.dat.gz
3
okay?y
preprocessed_habitat_data/train/PX4nDJXEHrG_082_4.dat.gz
preprocessed_habitat_data/DEMON/PX4nDJXEHrG_082_0.dat.gz
4
okay?y
preprocessed_habitat_data/train/PX4nDJXEHrG_243_1.dat.gz
preprocessed_habitat_data/DEMON/PX4nDJXEHrG_243_0.dat.gz
5
okay?y
preprocessed_habitat_data/train/VVfe2KiqLaN_205_0.dat.gz
preprocessed_habitat_data/DEMON/VVfe2KiqLaN_205_0.dat.gz
6
okay?y
preprocessed_habitat_data/train/gTV8FGcVJC9_243_1.dat.gz
preprocessed_habitat_data/DEMON/gTV8FGcVJC9_243_0.dat.gz
7
okay?y
preprocessed_hab

okay?y
preprocessed_habitat_data/train/mJXqzFtmKg4_033_1.dat.gz
preprocessed_habitat_data/DEMON/mJXqzFtmKg4_033_0.dat.gz
67
okay?n
preprocessed_habitat_data/train/mJXqzFtmKg4_163_3.dat.gz
preprocessed_habitat_data/DEMON/mJXqzFtmKg4_163_0.dat.gz
68
okay?n
preprocessed_habitat_data/train/s8pcmisQ38h_041_4.dat.gz
preprocessed_habitat_data/DEMON/s8pcmisQ38h_041_0.dat.gz
69
okay?y
preprocessed_habitat_data/train/gZ6f7yhEvPG_157_4.dat.gz
preprocessed_habitat_data/DEMON/gZ6f7yhEvPG_157_0.dat.gz
70
okay?n
preprocessed_habitat_data/train/jh4fc5c5qoQ_238_4.dat.gz
preprocessed_habitat_data/DEMON/jh4fc5c5qoQ_238_0.dat.gz
71
okay?y
preprocessed_habitat_data/train/PX4nDJXEHrG_178_1.dat.gz
preprocessed_habitat_data/DEMON/PX4nDJXEHrG_178_0.dat.gz
72
okay?y
preprocessed_habitat_data/train/mJXqzFtmKg4_084_2.dat.gz
preprocessed_habitat_data/DEMON/mJXqzFtmKg4_084_0.dat.gz
73
okay?y
preprocessed_habitat_data/train/jh4fc5c5qoQ_006_3.dat.gz
preprocessed_habitat_data/DEMON/jh4fc5c5qoQ_006_0.dat.gz
74
okay?y
p

okay?y
preprocessed_habitat_data/train/s8pcmisQ38h_259_4.dat.gz
preprocessed_habitat_data/DEMON/s8pcmisQ38h_259_0.dat.gz
133
okay?y
preprocessed_habitat_data/train/8WUmhLawc2A_284_3.dat.gz
preprocessed_habitat_data/DEMON/8WUmhLawc2A_284_0.dat.gz
134
okay?y
preprocessed_habitat_data/train/i5noydFURQK_260_3.dat.gz
preprocessed_habitat_data/DEMON/i5noydFURQK_260_0.dat.gz
135
okay?y
preprocessed_habitat_data/train/SN83YJsR3w2_289_2.dat.gz
preprocessed_habitat_data/DEMON/SN83YJsR3w2_289_0.dat.gz
136
okay?y
preprocessed_habitat_data/train/8WUmhLawc2A_053_3.dat.gz
preprocessed_habitat_data/DEMON/8WUmhLawc2A_053_0.dat.gz
137
okay?y
preprocessed_habitat_data/train/jh4fc5c5qoQ_027_2.dat.gz
preprocessed_habitat_data/DEMON/jh4fc5c5qoQ_027_0.dat.gz
138
okay?y
preprocessed_habitat_data/train/VVfe2KiqLaN_291_0.dat.gz
preprocessed_habitat_data/DEMON/VVfe2KiqLaN_291_0.dat.gz
139
okay?y
preprocessed_habitat_data/train/VVfe2KiqLaN_154_0.dat.gz
preprocessed_habitat_data/DEMON/VVfe2KiqLaN_154_0.dat.gz
140


KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

In [30]:
len(error_list)

201

In [1]:
#for t in error_list:
for j in range(4):
    [ref_rgb, ref_action, noise_rgb, noise_action] = load_data('preprocessed_habitat_data/train/b8cTxDM8gDG_008_{}.dat.gz'.format(j))
    for i in range(max(len(ref_rgb), len(noise_rgb))):
        view_image = np.zeros([256, 256*2, 3])
        view_image[:,:256,:] = (ref_rgb[i]+1)/2. if i < len(ref_rgb) else (ref_rgb[len(ref_rgb)-1]+1)/2.
        view_image[:,256:,:] = (noise_rgb[i]+1)/2. if i < len(noise_rgb) else (noise_rgb[len(noise_rgb)-1]+1)/2.
        cv2.imshow('v',view_image)
        cv2.waitKey(100)
    key = input('okay?')
    #if 'key' == 'y' : continue
    #else : print('hey@')

NameError: name 'load_data' is not defined